In [1]:
import collections
collections.Callable = collections.abc.Callable
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup, NavigableString
import time
import csv
import os

In [ ]:
#!pip install selenium          

In [2]:
#!pip install selenium webdriver-manager pandas

In [3]:
import selenium
selenium.__version__


'4.19.0'

In [4]:
relative_path_plant = "../India_Flora_13000/extracted_data_india_flora_1_13122.csv"
relative_path_med_data = "./extracted_med_data_4001_5000.csv"

absolute_path_plant = os.path.join(os.getcwd(), relative_path_plant)
absolute_path_med_data = os.path.join(os.getcwd(), relative_path_med_data)

In [5]:
plant_data = pd.read_csv(absolute_path_plant, index_col=0)

In [6]:
plant_data

,Plant_Id,Scientific_Name,Family,Family_Hindi_Name),Species_Name,Common_Name,Habit,Habitat,Comments,Flower_Fruit,Distribution,Key_Feature
0,1,Abelia chinensis,CAPRIFOLIACEAE(Honeysuckle Family),RASHINA FAMILY,Linnaea chinensis (R.Br.) A.Braun & Vatke,NaN,NaN,Cultivated,NaN,NaN,Sikkim,"An upright-rounded, multi-stemmed, deciduous s..."
1,2,Abelia triflora,CAPRIFOLIACEAE(Honeysuckle Family),RASHINA FAMILY,Zabelia triflora (R.Br. ex Wall.) Makino,Himalayan Abelia,Shrub,On dry rocky ground in Himalayas; 1500 - 4200 ...,NaN,May-June,Uttarakhand,NaN
2,3,Abelmoschus angulosus,MALVACEAE(Mallow Or Hibiscus Family),GURHAL FAMILY (गुड़हल फैमिली),Abelmoschus angulosus Wall. ex Wight & Arn.,NaN,Undershrub,"Tropical moist deciduous forests, Tropical sem...",An extremely variable species with regard to s...,Throughout the year,"Karnataka, Kerala, Maharashtra, Tamil Nadu",NaN
3,4,Abelmoschus angulosus,MALVACEAE(Mallow Or Hibiscus Family),GURHAL FAMILY (गुड़हल फैमिली),Abelmoschus angulosus Wall. ex Wight & Arn.,NaN,Undershrub,"Tropical moist deciduous forests, Tropical eve...",NaN,August-December,Kerala,NaN
4,5,Abelmoschus angulosus,MALVACEAE(Mallow Or Hibiscus Family),GURHAL FAMILY (गुड़हल फैमिली),Abelmoschus angulosus Wall. ex Wight & Arn.,NaN,Undershrub,"Grasslands, Shola",NaN,October-December,Kerala,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1117,13117,Zornia quilonensis,FABACEAE(Pea Or Bean Family),PALASH FAMILY (पलाश फैमिली),Zornia quilonensis Ravi,NaN,Herb,Plains,NaN,October-January,Kerala,NaN
1118,13118,Zosima anethifolia,APIACEAE(Carrot Or Coriander Or Parsley Family),SAUNF FAMILY (सौंफ फैमिली),Ducrosia anethifolia (DC.) Boiss.,NaN,Herb,Growing on the edges of cultivated fields,NaN,October-December,Gujarat,NaN
1119,13119,Zoysia matrella,POACEAE(Grass Family),GHAS FAMILY (घास फैमिली),Zoysia matrella (L.) Merr.,NaN,Herb,"Banks fo backwaters, streams and lakes and man...",Often as green patches on sandy soil.,March-September,"Kerala, Maharashtra, Odisha, Tamil Nadu",NaN
1120,13120,Zoysia matrella,POACEAE(Grass Family),GHAS FAMILY (घास फैमिली),Zoysia matrella var. pacifica Goudsw.,NaN,Herb,NaN,Cultivated extensively as lawn grass.,Throughout the year,NaN,NaN


In [7]:
def initialize_driver():
    # Initialize the Chrome webdriver using ChromeDriverManager
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    return driver

In [8]:
def perform_search(driver, pid, plant_name):
    # Navigate to the website
    data = []
    url = "https://cb.imsc.res.in/imppat/basicsearch/therapeutics"
    driver.get(url)
    
    # Wait for the page to load completely (adjust sleep time as needed)
    time.sleep(4)
    # Find the search input field and enter the search query
    search_query = plant_name
    search_input = driver.find_element(By.ID,"edit-combine")
    search_input.send_keys(search_query)

    # Submit the form
    search_input.submit()

    # Wait for the page to load completely (adjust sleep time as needed)
    #time.sleep(5)

    try:
        # Wait for the dropdown to load (indicating results are present)
        dropdown = WebDriverWait(driver, 4).until(
            EC.presence_of_element_located((By.NAME, "table_id_length"))
        )
    except Exception as e:
        #print(f"No results found {plant_name}")
        return data

    # Change the dropdown selection to 100 entries
    dropdown.click()
    driver.find_element(By.XPATH, "//option[text()='100']").click()

    # Wait for the page to reload with 100 entries per page
    time.sleep(4)
    
    # Get the HTML content of the page after search and setting 100 entries per page
    html_content = driver.page_source

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")

    # Find the search results table
    search_results_table = soup.find("table", {"class": "table"})

    if search_results_table:
        # Extract and print the data from each page
        page_number = 1
        while True:
            rows = search_results_table.find_all("tr")
            for row in rows:
                cells = row.find_all("td")
                if cells:
                    col1 = cells[0].text.strip()
                    col2 = cells[1].text.strip()
                    col3 = cells[2].text.strip()
                    data.append([pid, plant_name, col1, col2, col3])
                    #print(data)

            # Check if there's a Next button and if it's disabled, stop the loop
            next_button = driver.find_element(By.ID, "table_id_next") if driver.find_elements(By.ID, "table_id_next") else None
            if next_button and "disabled" in next_button.get_attribute("class"):
                #print(f"Reached the last page. Total pages visited: {page_number}")
                break
            elif next_button:
                next_button.click()
                time.sleep(5)  # Wait for the next page to load
                page_number += 1
                html_content = driver.page_source
                soup = BeautifulSoup(html_content, "html.parser")
                search_results_table = soup.find("table", {"class": "table"})
            else:
                #print("No more pages to load.")
                break
    else:
        print(f"No results found {plant_name}")
    
    return data

In [9]:
def search_and_store_results(plants_df):
    # Initialize the Chrome webdriver
    driver = initialize_driver()
    
    all_data = []
    name_list = []
    
    try:
        # Iterate over each row in the plants dataframe
        for index, row in plants_df.iterrows():
            plant_id = row["Plant_Id"]
            scientific_name = row["Scientific_Name"]
            if type(scientific_name) is str:
                plant_sci_name = " ".join(scientific_name.strip().split())
                if plant_sci_name not in name_list:
                    # Perform the search for the current plant name
                    one_plant = perform_search(driver, plant_id, plant_sci_name)
                    if len(one_plant) > 0:
                        all_data.extend(one_plant)
                    name_list.append(plant_sci_name)
    except Exception as e:
        print(e)
        return all_data
    # Close the webdriver
    driver.quit()
    
    return all_data

In [10]:
plant_data_100 = plant_data.iloc[4000:5000].copy()
plant_data_100

,Plant_Id,Scientific_Name,Family,Family_Hindi_Name),Species_Name,Common_Name,Habit,Habitat,Comments,Flower_Fruit,Distribution,Key_Feature
1,4001,Didymocarpus pedicellatus,GESNERIACEAE(Gesneria Family),CHARELA FAMILY,Didymocarpus pedicellatus R.Br.,Stone Flower,Herb,Shady moist banks and damp rock; 600 - 2500 m....,"Stone Flower is a valuable, although a lesser ...",June-September,"Uttarakhand, West Bengal","Leaves (also stems, which are very short) cove..."
2,4002,Didymocarpus primulifolius,GESNERIACEAE(Gesneria Family),CHARELA FAMILY,Didymocarpus primulifolius D.Don,Primrose-Leaved Stone Flower,Herb,Rocks or cliffs in forests of Himalaya; 2000 -...,NaN,June-July,Uttarakhand,NaN
3,4003,Didymocarpus pygmaeus,GESNERIACEAE(Gesneria Family),CHARELA FAMILY,Didymocarpus pygmaeus C.B.Clarke,NaN,Herb,On moss covered rock on hills,Rare,August-January,"Andhra Pradesh, Odisha",Single sessile leaf
4,4004,Didymocarpus repens,GESNERIACEAE(Gesneria Family),CHARELA FAMILY,Henckelia repens (Bedd.) A.Weber & B.L.Burtt,NaN,Herb,Tropical evergreen forests,NaN,May-December,Kerala,NaN
5,4005,Didymocarpus subalternans,GESNERIACEAE(Gesneria Family),CHARELA FAMILY,Unresolved name,NaN,Herb,NaN,NaN,NaN,Karnataka,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
996,4996,Euphorbia barnhartii,EUPHORBIACEAE(Spurge Family),AMLA FAMILY (आँवला फैमिली),Euphorbia lacei Craib,NaN,Herb,Dry hills,NaN,December-February,Andhra Pradesh,NaN
997,4997,Euphorbia caducifolia,EUPHORBIACEAE(Spurge Family),AMLA FAMILY (आँवला फैमिली),Euphorbia caducifolia Haines,NaN,Shrub,NaN,"Common in scrub jungles, wastelands and rocky ...",January-May,"Andhra Pradesh, Karnataka, Odisha",NaN
998,4998,Euphorbia cashmeriana,EUPHORBIACEAE(Spurge Family),AMLA FAMILY (आँवला फैमिली),Euphorbia cashmeriana Royle,NaN,Herb,NaN,NaN,NaN,Uttarakhand,NaN
999,4999,Euphorbia cattimandoo,EUPHORBIACEAE(Spurge Family),AMLA FAMILY (आँवला फैमिली),Euphorbia cattimandoo Elliot ex Wight,NaN,Shrub,NaN,NaN,January-February,Andhra Pradesh,NaN


In [22]:
# Assuming you have a dataframe called plants_df with columns "plant_id" and "Scientific_Name"
results_list = search_and_store_results(plant_data_100)

In [23]:
results_list

[[3503,
  'Cymbidium iridioides',
  'Cymbidium iridioides',
  '',
  'Parasympatholytics'],
 [3509,
  'Cymbopogon caesius',
  'Cymbopogon caesius',
  'whole plant',
  'Anti-bacterial agents'],
 [3509,
  'Cymbopogon caesius',
  'Cymbopogon caesius',
  '',
  'Anti-bacterial agents'],
 [3509,
  'Cymbopogon caesius',
  'Cymbopogon caesius',
  '',
  'Central nervous system diseases'],
 [3509, 'Cymbopogon caesius', 'Cymbopogon caesius', '', 'Parasympatholytics'],
 [3510,
  'Cymbopogon citratus',
  'Cymbopogon citratus',
  'fruit',
  'Antiviral agents'],
 [3510, 'Cymbopogon citratus', 'Cymbopogon citratus', 'leaf', 'Analgesia'],
 [3510, 'Cymbopogon citratus', 'Cymbopogon citratus', 'leaf', 'Analgesics'],
 [3510,
  'Cymbopogon citratus',
  'Cymbopogon citratus',
  'leaf',
  'Anti-bacterial agents'],
 [3510,
  'Cymbopogon citratus',
  'Cymbopogon citratus',
  'leaf',
  'Antifungal agents'],
 [3510,
  'Cymbopogon citratus',
  'Cymbopogon citratus',
  'leaf',
  'Antinematodal agents'],
 [3510,
  '

In [24]:
len(results_list)

2671

In [25]:
# Create an empty dataframe to store the results
results_df = pd.DataFrame(results_list, columns=["Plant_id", "Scientific_name", "Plant_Name_Mapped", "Plant_Part", "Therapeutic_Use"])

In [26]:
results_df

,Plant_id,Scientific_name,Plant_Name_Mapped,Plant_Part,Therapeutic_Use
0,3503,Cymbidium iridioides,Cymbidium iridioides,,Parasympatholytics
1,3509,Cymbopogon caesius,Cymbopogon caesius,whole plant,Anti-bacterial agents
2,3509,Cymbopogon caesius,Cymbopogon caesius,,Anti-bacterial agents
3,3509,Cymbopogon caesius,Cymbopogon caesius,,Central nervous system diseases
4,3509,Cymbopogon caesius,Cymbopogon caesius,,Parasympatholytics
...,...,...,...,...,...
2666,3983,Dictamnus albus,Dictamnus albus,,Anthelmintics
2667,3983,Dictamnus albus,Dictamnus albus,,Parasympatholytics
2668,3983,Dictamnus albus,Dictamnus albus,,Scabies
2669,3983,Dictamnus albus,Dictamnus albus,,Skin diseases


In [27]:
results_df.to_csv(absolute_path_med_data, index=False)

In [13]:
# # selenium 4
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service as ChromeService
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.by import By
# from webdriver_manager.chrome import ChromeDriverManager
# from bs4 import BeautifulSoup
# import pandas as pd
# import time

# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# # Navigate to the website
# url = "https://cb.imsc.res.in/imppat/basicsearch/therapeutics"
# driver.get(url)

# # Wait for the page to load completely (adjust sleep time as needed)
# time.sleep(4)

# # Find the search input field and enter the search query
# search_query = "adkjnwkdjn"
# search_input = driver.find_element(By.ID,"edit-combine")
# search_input.send_keys(search_query)

# # Submit the form
# search_input.submit()

# # # Wait for the page to load completely (adjust sleep time as needed)
# # time.sleep(5)

# try:
#     # Wait for the dropdown to load (indicating results are present)
#     dropdown = WebDriverWait(driver, 5).until(
#         EC.presence_of_element_located((By.NAME, "table_id_length"))
#     )
# except Exception as e:
#     print(f"No results found")
#     driver.quit()

# # Change the dropdown selection to 100 entries
# dropdown.click()
# driver.find_element(By.XPATH, "//option[text()='100']").click()

# # Wait for the page to reload with 100 entries per page
# time.sleep(5)

# # Get the HTML content of the page after search and setting 100 entries per page
# html_content = driver.page_source

# # Parse the HTML content using BeautifulSoup
# soup = BeautifulSoup(html_content, "html.parser")

# # Find the search results table
# search_results_table = soup.find("table", {"class": "table"})

# if search_results_table:
#     # Extract and print the data from each page
#     page_number = 1
#     while True:
#         rows = search_results_table.find_all("tr")
#         for row in rows:
#             cells = row.find_all("td")
#             if cells:
#                 data = cells[0].text.strip()
#                 print(data)

#         # Check if there's a Next button and if it's disabled, stop the loop
#         next_button = driver.find_element(By.ID, "table_id_next") if driver.find_elements(By.ID, "table_id_next") else None
#         if next_button and "disabled" in next_button.get_attribute("class"):
#             print(f"Reached the last page. Total pages visited: {page_number}")
#             break
#         elif next_button:
#             next_button.click()
#             time.sleep(5)  # Wait for the next page to load
#             page_number += 1
#             html_content = driver.page_source
#             soup = BeautifulSoup(html_content, "html.parser")
#             search_results_table = soup.find("table", {"class": "table"})
#         else:
#             print("No more pages to load.")
#             break
# else:
#     print("Search results table not found.")

# # Close the webdriver
# driver.quit()

No results found


MaxRetryError: HTTPConnectionPool(host='localhost', port=44207): Max retries exceeded with url: /session/cc697801d382f00c9fb6714d5d63ec70/element/2BD685820D4BA137066FADF9A36B6F99_element_32/click (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7216e336b580>: Failed to establish a new connection: [Errno 111] Connection refused'))